In [1]:
import torch

In [2]:
words=open('/home/maria/andrej-karpathy-experiments/names.txt','r').read().splitlines()

In [13]:
def generate_bigrams(words):
    # create the training set of bigrams (x,y)
    xs, ys = [], []
    for w in words:
      chs = ['.'] + list(w) + ['.']
      for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

    xs = torch.tensor(xs).to('cuda')
    ys = torch.tensor(ys).to('cuda')
    return xs, ys

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
xs, ys = generate_bigrams(words)

In [25]:
xs.shape[0]

228146

In [32]:
#Network
generator=torch.Generator(device='cuda').manual_seed(1337)
def nn_forward_pass(x, W):
    x_enc=torch.nn.functional.one_hot(x,len(stoi)).float()
    logits=x_enc@W
    probs=logits.exp()/logits.exp().sum(1,keepdim=True)
    return probs

def loss(probs, y, alpha, W):
    nll=-probs[range(len(y)),y].log().mean()+alpha*(W**2).mean()
    return nll

def optimize_single_hyperparam(alpha):
    train_x=xs[:int(xs.shape[0]*0.8)]
    train_y=ys[:int(ys.shape[0]*0.8)]
    valid_x=xs[int(xs.shape[0]*0.8):]
    valid_y=ys[int(ys.shape[0]*0.8):]
    generator=torch.Generator(device='cuda').manual_seed(1337)
    #parameters
    W=torch.randn((len(stoi),len(stoi)),requires_grad=True, generator=generator, device='cuda')
    for i in range(1000):
        probs=nn_forward_pass(train_x,W)
        nll=loss(probs,train_y, alpha,W)
        W.grad=None
        nll.backward()
        W.data-=W.grad*1.0
    validation_loss=loss(nn_forward_pass(valid_x,W),valid_y, 0, W)
    return validation_loss


alpha=[0.0001,0.001,0.01,0.1,1,10,100]

for a in alpha:
    print(optimize_single_hyperparam(a))

tensor(2.7073, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7072, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7063, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7026, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.8166, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1587, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2795, device='cuda:0', grad_fn=<AddBackward0>)
